In [0]:
from pyspark.sql.functions import col

df_stream = (
    spark.readStream
    .format("cloudFiles")  
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option(
        "cloudFiles.schemaLocation",
        "/Volumes/workspace/product/productt/schema/autoloader"
    ) 
    .option(
        "cloudFiles.schemaEvolutionMode",
        "addNewColumns"
    )  
    .load("/Volumes/workspace/product/productt/incoming_autoloader")
)


In [0]:
df_clean = df_stream.toDF(
    *[c.lower().replace(" ", "_") for c in df_stream.columns]
)


In [0]:
df_clean = df_clean.filter(
    col("price").isNotNull() &
    col("stock").isNotNull()
)


In [0]:
query = (
    df_clean.writeStream
    .format("delta")
    .outputMode("append")
    .option(
        "checkpointLocation",
        "/Volumes/workspace/product/productt/_checkpoints/autoloader"
    )
    .trigger(availableNow=True)
    .start("/Volumes/workspace/product/productt/delta/silver")
)

In [0]:
silver_df = spark.read.format("delta") \
    .load("/Volumes/workspace/product/productt/delta/silver")

display(silver_df)


In [0]:
spark.read.format("delta") \
  .load("/Volumes/workspace/product/productt/delta/silver") \
  .printSchema()


In [0]:
silver_df = spark.read.format("delta") \
    .load("/Volumes/workspace/product/productt/delta/silver")

silver_df.count()
